In [3]:
import numpy as np # thư viện tính toán 
import pandas as pd # đọc file csv
import concurrent.futures as cf # thư viện giúp code python chạy đa luồng
from hmmlearn import hmm # thư viện mô hình Markov ẩn 
from sklearn.cluster import KMeans # lượng hóa vector
from sklearn.metrics import accuracy_score # đo độ chính xác của mô hình
from sklearn.decomposition import TruncatedSVD # giảm chiều dữ liệu
from sklearn.model_selection import train_test_split # chia tập dữ liệu train|test
from sklearn.feature_extraction.text import TfidfVectorizer # tạo feature cho mô hình từ chữ

In [4]:
rs = 8

In [5]:
df = pd.read_csv('all-data.csv', encoding="ISO-8859-1", header=None, names=['label', 'text'])
df

,label,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [6]:
corpus = df['text'].values

### 1. Tạo feature dữ liệu số từ dữ liệu chữ có sẵn bằng TF-IDF và giữ thông tin quan trọng bằng Truncated SVD

In [37]:
tfidf = TfidfVectorizer(stop_words='english')
transformed = tfidf.fit_transform(corpus)

print("Kích cỡ dữ liệu:", transformed.shape)

Kích cỡ dữ liệu: (4846, 9820)


In [39]:
svd = TruncatedSVD(n_components=300, random_state=rs)
X_transformed = svd.fit_transform(transformed)

print("Kích cỡ dữ liệu:", X_transformed.shape)

Kích cỡ dữ liệu: (4846, 300)


In [44]:
print(X_transformed)

[[ 2.74853772e-02  1.23546023e-01 -9.18267054e-02 ... -1.04585446e-02
   6.31456379e-02  1.64352977e-02]
 [ 1.71779475e-02  6.60849287e-02 -3.16730100e-02 ...  3.16069539e-02
   2.32036018e-02 -8.03645790e-03]
 [ 2.53099565e-02  8.71819162e-02 -5.00550792e-02 ... -5.08493999e-02
  -6.52592321e-02 -4.16690704e-02]
 ...
 [ 6.29146372e-01 -1.92710754e-01  3.24730118e-02 ...  3.28086253e-02
  -1.07719957e-02 -8.53337727e-04]
 [ 6.66497950e-01 -1.41546038e-01  1.81541966e-03 ...  7.96125651e-03
  -2.97791037e-03 -1.60182069e-04]
 [ 9.54134895e-02  1.71111951e-01 -6.08651229e-02 ... -1.30032385e-02
   3.79181183e-02  1.24508153e-02]]


### 2. Lượng tử vector (vector quantization) thành dạng định tính

In [9]:
X_cluster = X_transformed.reshape(-1, 1)

In [10]:
%%time

vq = KMeans(n_clusters=30) # vector quantization
vq.fit(X_cluster)

CPU times: user 1min 57s, sys: 5.19 s, total: 2min 2s
Wall time: 1min 32s


KMeans(n_clusters=30)

In [11]:
%%time

def map_vq(x):
    return vq.predict(x.reshape(-1, 1))

with cf.ThreadPoolExecutor() as exe:
    X = np.array(list(exe.map(map_vq, X_transformed)))

CPU times: user 3.87 s, sys: 2.46 s, total: 6.33 s
Wall time: 3.13 s


In [45]:
print(X)

[[ 9 11  0 ...  8 16 22]
 [27 16 20 ...  9 27  8]
 [ 9  3 19 ... 19 28 10]
 ...
 [26 24  4 ...  4  8 13]
 [26 24 13 ...  1 29 13]
 [ 3 18 28 ... 17  4 22]]


### 3. Phân chia tập train test

In [12]:
y = df['label']

In [13]:
labels = np.unique(y)
labels

array(['negative', 'neutral', 'positive'], dtype=object)

In [14]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=rs)

### 4. Huấn luyện mô hình Markov ẩn

In [41]:
class HMMSystem:
    def __init__(self, n_components=10, random_state=rs):
        self.n_components = n_components
        self.random_state = random_state
    
    def fit(self, X, y):
        self.labels = np.unique(y)
        self.X = X
        self.y = y
        self.hmm_models = {}
        for c in self.labels:
            with cf.ThreadPoolExecutor() as exe:
                self.hmm_models = list(exe.map(self._create_model, self.labels))
            self.hmm_models = dict(zip(self.labels, self.hmm_models))
        return self
        
    def predict(self, X):
        with cf.ThreadPoolExecutor() as exe:
            pred = np.array(list(exe.map(self._find_class, X)))
        return pred
    
    def _create_model(self, label):
        model = hmm.MultinomialHMM(
            n_components=self.n_components,
            random_state=self.random_state
        ).fit(self.X[self.y == label])
        return model
    
    def _find_class(self, data):
        def _decode(model):
            return model.decode([data])[0]

        with cf.ThreadPoolExecutor() as exe:
            logprobs = list(exe.map(_decode, self.hmm_models.values()))

        return self.labels[np.argmax(logprobs)]

In [48]:
%%time

model = HMMSystem(8, rs)
model.fit(X, y)

CPU times: user 17min 26s, sys: 5.09 s, total: 17min 31s
Wall time: 8min 20s


In [49]:
%%time

ytest_pred = model.predict(Xtest)
ytrain_pred = model.predict(Xtrain)

CPU times: user 10.5 s, sys: 2.75 s, total: 13.2 s
Wall time: 9.47 s


In [50]:
acc_train = accuracy_score(ytrain, ytrain_pred)
acc_test = accuracy_score(ytest, ytest_pred)

print("Độ chính xác tập huấn luyện: %.2f/1" % acc_train)
print("Độ chính xác tập kiểm thử: %.2f/1" % acc_test)

Độ chính xác tập huấn luyện: 0.46/1
Độ chính xác tập kiểm thử: 0.48/1
